# Dados

In [10]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import cv2
import shutil
from module import data_read, setup
from torch.utils.data import ConcatDataset
from module.LightningModule import ClassificationModule
from module.setup import *
from module.data_read import *

torch.manual_seed(random_state)

DDR_DIR = Path.cwd() / 'DDR-dataset/DR_grading'
IDRID_DIR = Path.cwd() / 'IDRID-Classificacao'
#FGADR_DIR = Path.cwd() / 'FGADR Dataset/Seg-set'

## Binary

In [19]:
# Leitura DDR
train_ddr_dataset = DDRDataset(DDR_DIR/'train',
                           DDR_DIR/'train.txt',           
                           convert_to_binary=True)
valid_ddr_dataset = DDRDataset(DDR_DIR/'valid',
                           DDR_DIR/'valid.txt',         
                           convert_to_binary=True)
test_ddr_dataset = DDRDataset(DDR_DIR/'test',
                          DDR_DIR/'test.txt',          
                          convert_to_binary=True)

ddr_dataset = ConcatDataset([train_ddr_dataset, valid_ddr_dataset, test_ddr_dataset])



# Leitura IDRID
train_idrid_dataset = IDRIDDataset(IDRID_DIR/'1. Original Images/a. Training Set',
                                   IDRID_DIR/'2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv',
                                   data_transforms['test'],
                                   convert_to_binary=True)
test_idrid_dataset = IDRIDDataset(IDRID_DIR/'1. Original Images/b. Testing Set',
                                  IDRID_DIR/'2. Groundtruths/b. IDRiD_Disease Grading_Testing Labels.csv',                   
                                  data_transforms['test'],
                                  convert_to_binary=True)
idrid_dataset = ConcatDataset([train_idrid_dataset, test_idrid_dataset])

##### EfficientNet-b7

In [ ]:
from torchvision import models
from torch import nn

# Instanciação EfficientNetB7
effcb7_best_weights = models.EfficientNet_B7_Weights.DEFAULT
effcb7_model = models.efficientnet_b7(weights=effcb7_best_weights)
effcb7_preprocess = effcb7_best_weights.transforms()

Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-c5b4e57e.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b7_lukemelas-c5b4e57e.pth
100%|██████████| 255M/255M [00:05<00:00, 50.6MB/s]


In [ ]:
effcb7_model